In [ ]:
#First, we import all the necessary library.This neural network is implemented in keras.
#Next we will import all the module that we'll use through out this notebook
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from skimage.color import gray2rgb,rgb2gray,label2rgb
import cv2 as cv
import numpy as np
import os
from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
from keras.models import Sequential, load_model
from sklearn.metrics import accuracy_score
import seaborn as sns 
img_size=224


In [ ]:
directory=['dyed-lifted-polyps','dyed-resection-margins','esophagitis','normal-cecum','normal-pylorus','normal-z-line','polyps','ulcerative-colitis']
y=[]
for i in directory:
    print(i)
    y.append(len(os.listdir(os.path.join('../input/dataset/kvasir-dataset-v2' +'/' +i))))
print(y)                  

In [ ]:
!pip install split-folders
input_folder = '../input/dataset/kvasir-dataset-v2'
import splitfolders

splitfolders.ratio(input_folder,output='data2',seed=42,ratio=(.9,.1),group_prefix=None)

Copying files: 8000 files [01:09, 115.04 files/s]


In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=30,
    data_format='channels_last',
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    validation_split=0.112
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
'./data2/train',
target_size=(img_size,img_size),
batch_size=16,
color_mode = 'rgb',
shuffle = True,
seed = 42,
class_mode='categorical',
subset = 'training'
)


valid_generator = train_datagen.flow_from_directory(
directory="./data2/train",
target_size=(img_size, img_size),
color_mode="rgb",
batch_size=16,
class_mode="categorical",
shuffle=True,
seed=42,
subset ='validation'

)


In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    # horizontal_flip=True,
    # zoom_range=[0.5, 1.25],
    rescale=1./255, 
 )

test_generator = test_datagen.flow_from_directory(
    directory="./data2/val",
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=1,
    class_mode='binary',
    shuffle=False,
    seed=42
    )

In [ ]:
import tensorflow_addons as tfa
#installing vit-keras package
!pip install vit-keras
from vit_keras import vit

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D,Conv2D,MaxPooling2D,Input,Dense,Flatten,Dropout,Activation,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

vit_model = vit.vit_l16(
         image_size = 224,
         activation = 'softmax',
         pretrained = True,
         include_top = True,
         pretrained_top = False,
        classes =8)

In [ ]:
def createmodel():
    model = Sequential()
    model.add(vit_model) #loads above imported pretrained model
    # # here we added new layers
    model.add(BatchNormalization())
    model.add(Dense(512, activation='gelu',kernel_initializer='lecun_normal'))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='softmax', name ='output'))
  
    opt = Adam(lr= 1e-3/1000, decay= 1e-3 /1000)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

In [ ]:
model=createmodel()
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
my_callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min',min_delta=1e-4)]

In [ ]:
history=model.fit (
        train_generator,
        epochs=1,
        validation_data=valid_generator,
        validation_steps=20
        )

In [ ]:
model.save('ksavir0.hdf5')

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/ksavir0.hdf5')

prediction=model.predict(test_generator,verbose=1)
print(prediction)

In [ ]:
yp=np.argmax(prediction,axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,matthews_corrcoef
print('Classification Report')

print(classification_report(test_generator.classes, yp, target_names=directory,digits=4))
print(matthews_corrcoef(test_generator.classes,yp))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()